# MARATONA BEHIND THE CODE 2020

## DESAFIO 2: PARTE 1

### Trabalhando com Pipelines do scikit-learn

In [34]:
# Primeiro, realizamos a instalação do scikit-learn versão 0.20.0 no Kernel deste notebook:
!pip install scikit-learn==0.20.0 --upgrade

Requirement already up-to-date: scikit-learn==0.20.0 in /opt/conda/envs/Python36/lib/python3.6/site-packages (0.20.0)


In [57]:
# Em seguida iremos importar diversas bibliotecas que serão utilizadas:

import json
import requests
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, cross_validate

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score


### Importando um .csv de seu projeto no IBM Cloud Pak for Data para o Kernel deste notebook

In [36]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_2e29cc98c92249589fb271f5cf851e63 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='Fr6Rl6A4GcKbw34FaMIyWwrxwMYSyiRjXHlIBG4O2LO4',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

body = client_2e29cc98c92249589fb271f5cf851e63.get_object(Bucket='desafio2ueder-donotdelete-pr-8l3dh7p4ewnvvl',Key='dataset_desafio_2.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_data_1 = pd.read_csv(body)
df_data_1.head()


,MATRICULA,NOME,REPROVACOES_DE,REPROVACOES_EM,REPROVACOES_MF,REPROVACOES_GO,NOTA_DE,NOTA_EM,NOTA_MF,NOTA_GO,INGLES,H_AULA_PRES,TAREFAS_ONLINE,FALTAS,PERFIL
0,502375,Márcia Illiglener,0,0,0,0,6.2,5.8,4.6,5.9,0.0,2,4,3,EXATAS
1,397093,Jason Jytereoman Izoimum,0,0,0,0,6.0,6.2,5.2,4.5,1.0,2,4,3,EXATAS
2,915288,Bartolomeu Inácio da Gama,0,0,0,0,7.3,6.7,7.1,7.2,0.0,5,0,3,HUMANAS
3,192652,Fernanda Guedes,1,3,1,1,0.0,0.0,0.0,0.0,1.0,4,4,4,DIFICULDADE
4,949491,Alessandre Borba Gomes,1,3,1,1,0.0,0.0,0.0,0.0,1.0,5,2,5,DIFICULDADE


In [40]:
#print(df_data_3.groupby(['PERFIL']).count())

In [37]:
from sklearn.base import BaseEstimator, TransformerMixin

#delete columns
class DropColumns(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        # Primeiro realizamos a cópia do dataframe 'X' de entrada
        data = X.copy()
        # Retornamos um novo dataframe sem as colunas indesejadas
        return data.drop(labels=self.columns, axis='columns')

# Instanciando uma transformação DropColumns
rm_columns = DropColumns(
    columns=["MATRICULA","NOME","H_AULA_PRES","TAREFAS_ONLINE","FALTAS","REPROVACOES_DE","REPROVACOES_EM","REPROVACOES_MF","REPROVACOES_GO"]  # Essa transformação recebe como parâmetro uma lista com os nomes das colunas indesejadas
)
print(rm_columns)

# Visualizando as colunas do dataset original
print("Colunas do dataset original: \n")
print(df_data_1.columns)

# Aplicando a transformação ``DropColumns`` ao conjunto de dados base
rm_columns.fit(X=df_data_1)

# Reconstruindo um DataFrame Pandas com o resultado da transformação
df_data_2 = pd.DataFrame.from_records(
    data=rm_columns.transform(
        X=df_data_1
    ),
)

# Visualizando as colunas do dataset transformado
print("Colunas do dataset após a transformação ``DropColumns``: \n")
print(df_data_2.columns)

DropColumns(columns=['MATRICULA', 'NOME', 'H_AULA_PRES', 'TAREFAS_ONLINE', 'FALTAS', 'REPROVACOES_DE', 'REPROVACOES_EM', 'REPROVACOES_MF', 'REPROVACOES_GO'])
Colunas do dataset original: 

Index(['MATRICULA', 'NOME', 'REPROVACOES_DE', 'REPROVACOES_EM',
       'REPROVACOES_MF', 'REPROVACOES_GO', 'NOTA_DE', 'NOTA_EM', 'NOTA_MF',
       'NOTA_GO', 'INGLES', 'H_AULA_PRES', 'TAREFAS_ONLINE', 'FALTAS',
       'PERFIL'],
      dtype='object')
Colunas do dataset após a transformação ``DropColumns``: 

Index(['NOTA_DE', 'NOTA_EM', 'NOTA_MF', 'NOTA_GO', 'INGLES', 'PERFIL'], dtype='object')


In [38]:
# Criação de um objeto ``SimpleImputer``
si = SimpleImputer(
    missing_values=np.nan,  # os valores faltantes são do tipo ``np.nan`` (padrão Pandas)
    strategy='constant',  # a estratégia escolhida é a alteração do valor faltante por uma constante
    fill_value=0,  # a constante que será usada para preenchimento dos valores faltantes é um int64=0.
    verbose=0,
    copy=True
)

# Visualizando os dados faltantes do dataset após a primeira transformação (df_data_2)
print("Valores nulos antes da transformação SimpleImputer: \n\n{}\n".format(df_data_2.isnull().sum(axis = 0)))

# Aplicamos o SimpleImputer ``si`` ao conjunto de dados df_data_2 (resultado da primeira transformação)
si.fit(X=df_data_2)

# Reconstrução de um novo DataFrame Pandas com o conjunto imputado (df_data_3)
df_data_3 = pd.DataFrame.from_records(
    data=si.transform(
        X=df_data_2
    ),  # o resultado SimpleImputer.transform(<<pandas dataframe>>) é lista de listas
    columns=df_data_2.columns  # as colunas originais devem ser conservadas nessa transformação
)

# Visualizando os dados faltantes do dataset após a segunda transformação (SimpleImputer) (df_data_3)
print("Valores nulos no dataset após a transformação SimpleImputer: \n\n{}\n".format(df_data_3.isnull().sum(axis = 0)))

Valores nulos antes da transformação SimpleImputer: 

NOTA_DE       0
NOTA_EM       0
NOTA_MF       0
NOTA_GO    3716
INGLES     3628
PERFIL        0
dtype: int64

Valores nulos no dataset após a transformação SimpleImputer: 

NOTA_DE    0
NOTA_EM    0
NOTA_MF    0
NOTA_GO    0
INGLES     0
PERFIL     0
dtype: int64



In [24]:
#print(df_data_3["PERFIL"].unique())

In [39]:
X = df_data_3.drop("PERFIL", axis = 1)
y = df_data_3.PERFIL

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=337)

In [41]:
feats = ["NOTA_DE", "NOTA_EM", "NOTA_MF", "NOTA_GO"]


In [60]:
xgboost = XGBClassifier(
 base_score=0.5,
 booster='gbtree',
 colsample_bylevel=1,
 colsample_bynode=1,
 colsample_bytree=1,
 gamma=0,
 learning_rate=0.1,
 max_delta_step=0,
 max_depth=3,
 max_features='sqrt',
 min_child_weight=1,
 min_samples_split=2,
 missing=None,
 n_estimators=50,
 n_jobs=1,
 nthread=None,
 objective='multi:softprob',
 random_state=0,
 reg_alpha=0,
 reg_lambda=1,
 scale_pos_weight=1,
 seed=None,
 subsample=1,
)

xgboost.fit(X_train, y_train)

y_pred = xgboost.predict(X_test)


In [61]:
print("Acurácia: {}%".format(100*round(accuracy_score(y_test, y_pred), 2)))

Acurácia: 82.0%


In [32]:
#from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

#print (classification_report(y_test, y_pred))


In [27]:
#Confusion Matrix 
#print (pd.crosstab(y_test, y_pred, rownames=['Real'], colnames=['Predito'], margins=True))
